In [ ]:
# 2018A 2) 结果

In [1]:
import numpy as np

In [2]:
# 附件1. 专用服装材料的参数值				
# 分层	"密度
# (kg/m）"	"比热
# (J/(kg·ºC))"	"热传导率
# (W/(m·ºC))"	"厚度
# (mm)"
# I层	300	1,377	0.08	0.60
# II层	862	2,100	0.37	0.6-25
# III层	74.20	1,726	0.05	3.60
# IV层	1.18	1,005	0.03	0.6-6.4

In [3]:
time = 3600
dx = 0.01e-3
dt = 0.1
n_ = int(time // dt)

def cal(sec):
    rho = np.array([300, 862, 74.20, 1.18])
    c = np.array([1377, 2100, 1726, 1005])
    k = np.array([0.08, 0.37, 0.05, 0.03])
    t = np.array([0.6e-3, sec, 3.6e-3, 5.5e-3])
    b_ = [0]
    for each_t in t:
        b_.append(b_[-1] + int(each_t / dx))
    M_B = np.zeros((b_[-1] + 1, b_[-1] + 1))
    dt_over_dx_squared = dt / (dx ** 2)
    for part in range(1, len(b_)):
        ld = dt_over_dx_squared * k[part - 1] / (c[part - 1] * rho[part - 1])
        for x_ in range(b_[part - 1], b_[part]):
            M_B[x_][x_] = 1 + 2 * ld
            if x_ != 0:
                M_B[x_][x_ - 1] = -ld
            if x_ != b_[-1]:
                M_B[x_][x_ + 1] = -ld
    for part, border in enumerate(b_[1:-1]):
        nu_last = k[part]
        nu_next = k[part + 1]
        M_B[border][border] = nu_last + nu_next
        M_B[border][border - 1] = -nu_last
        M_B[border][border + 1] = -nu_next
    M_B[0][1] = -1
    M_B[-1][-2] = -1
    T_env = 65
    T_body = 37

    h0, h1 = 118.48722172, 8.99325409
    mu0 = dx * h0 / k[0]
    mu1 = dx * h1 / k[-1]
    M_B[0][0] = 1 + mu0
    M_B[-1][-1] = 1 + mu1
    u_0 = mu0 * T_env
    u_last = mu1 * T_body

    T = np.ndarray((n_, b_[-1] + 1))
    T.fill(T_body)
    mask = np.zeros_like(T[0], dtype=bool)
    mask[b_[1:-1]] = True

    M_B_inv = np.linalg.inv(M_B)

    for t_ in range(1, n_):
        # if t_ % int(n_ / 3) == 0:
        #     print(t_ / n_)
        equ_b = T[t_ - 1].copy()
        equ_b[0] = u_0
        equ_b[-1] = u_last
        equ_b[mask] = 0
        T[t_] = M_B_inv @ equ_b
    return T

In [4]:
T = cal(0.6e-3)

In [5]:
print(np.sum(T[:, -1] >= 44))
print((5 * 60) / dt)
print(T[-1][-1])

35301
3000.0
45.12053752584734


In [6]:
for each in np.arange(17.3e-3, 17.4e-3, 1e-5):
    T = cal(each)
    print(each)
    print(np.sum(T[:, -1] >= 44))
    print(np.sum(T[:, -1] >= 44) <= (5 * 60) / dt)
    print(T[-1][-1])
    print(T[-1][-1] <= 47)
    print("-"*20)

0.0173
3093
False
44.075128660866625
True
--------------------
0.01731
3050
False
44.07405824667399
True
--------------------
0.01732
3006
False
44.07298674588829
True
--------------------
0.017329999999999998
2963
True
44.0719141578339
True
--------------------
0.017339999999999998
2919
True
44.070840483198154
True
--------------------
0.017349999999999997
2876
True
44.06976572181902
True
--------------------
0.017359999999999997
2832
True
44.06868987405951
True
--------------------
0.017369999999999997
2788
True
44.06761293989731
True
--------------------


KeyboardInterrupt: 

In [ ]:
# 0.0174